In [102]:
import arcpy
import json
import pprint
import urllib
import ssl
import os
## import urllib2

In [103]:
#allow unverified SSL context
ssl._create_default_https_context = ssl._create_unverified_context

In [104]:
#initialize dictionary for metro counties from MnGeo
counties_dict = {
    'name': 'metro_counties',
    'notes': "Dataset from MnGeo for metro county and CTU boundaries"
}

In [105]:
#use JSON module to dump the dictionary to a string for posting
counties_string = urllib.request.quote(json.dumps(counties_dict))

In [106]:
#create a new dataset using package_create
request = urllib.request.Request('https://gisdata.mn.gov/api/action/package_create')

In [107]:
#authorize with API credentials
request.add_header('Authorization', '4d517974-6e8a-43e1-b031-d181ee8a702b')

In [108]:
#use JSON module to load reponse into a dictionary
response_dict = json.loads(response.read())
assert response_dict['success'] is True

In [109]:
#make HTTP request for group list
response = urllib.request.urlopen('https://gisdata.mn.gov/api/action/group_list')
assert response.code == 200

In [110]:
#make HTTP request for package search: boundaries
response = urllib.request.urlopen('https://gisdata.mn.gov/api/action/package_search?q=boundaries')
assert response.code == 200

In [111]:
#make HTTP request for package - county boundaries
response = urllib.request.urlopen('https://gisdata.mn.gov/api/action/package_show?id=d199886f-474f-4fe0-aba4-c7071ab92a35')
assert response.code == 200

In [112]:
#package_create returns the created package as its result
created_package = response_dict['result']
pprint.pprint(created_package)

{'author': None,
 'author_email': None,
 'creator_user_id': '61044ca7-ee56-4019-aef7-f7c3b5e06e3a',
 'extras': [{'key': 'dsAccessConst', 'value': 'None'},
            {'key': 'dsCurrentRef',
             'value': 'Data is published weekly to the GeoCommons, but maybe '
                      'maintained on a different cycle.  Open package to '
                      'review specific layer for more detail.'},
            {'key': 'dsMetadataUrl',
             'value': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_counties/metadata/metadata.html'},
            {'key': 'dsModifiedDate', 'value': '2021-09-30 00:02:30'},
            {'key': 'dsOriginator',
             'value': 'Minnesota Department of Transportation'},
            {'key': 'dsPurpose',
             'value': 'The purpose of this dataset is to spatially represent '
                      'the county boundaries and use as reference for other '
                      'layers spatially dependent on county b

In [113]:
arcpy.management.CreateFileGDB(r"C:\Users\Holly Leaf\GIS5571\Lab1\GIS 5571 Lab 1", "Lab1.gdb")
arcpy.env.workspace = r"C:\Users\Holly Leaf\GIS5571\Lab1\GIS 5571 Lab 1/Lab1.gbd"

In [116]:
#RESULTS IN ERROR - How to get from JSON to imported Feature Class
#Until error is resolved, features will be added using the ArcGIS GUI to continue lab requirements
arcpy.JSONToFeatures_conversion(created_package, os.path.join("lab1.gdb","myfeatures"))

RuntimeError: Object: Error in executing tool

In [128]:
#ArcPy check if feature classes use same coordinate system
fc1 = 'my_neighborhood_sites'
fc2 = 'Census2020Counties'
sr1 = arcpy.Describe(fc1).spatialReference
sr2 = arcpy.Describe(fc2).spatialReference
sr1.name == sr2.name

True

In [129]:
#ArcPy spatial join
#metro counties joined with MPCA sites
arcpy.analysis.SpatialJoin('my_neighborhood_sites', 'Census2020Counties', 'MPCA_in_metro_counties', 'JOIN_ONE_TO_MANY', 'KEEP_COMMON', field_mapping = 'count', match_option="WITHIN")

<Result 'C:\\Users\\Holly Leaf\\GIS5571\\Lab1\\GIS 5571 Lab 1\\GIS 5571 Lab 1.gdb\\MPCA_in_metro_counties'>

In [130]:
#print merged attribute headers
desc = arcpy.Describe("MPCA_in_metro_counties")
for field in desc.fields:
    print(field.name)

OBJECTID
Shape
Join_Count
TARGET_FID
JOIN_FID
count
